In [2]:
import pandas as pd
import matplotlib as plt
import numpy as np
import seaborn as sns

In [3]:
!pip install scipy

In [4]:
!pip install yfinance

In [5]:
import scipy as sp
import yfinance as yf

In [1]:
import platform

In [2]:
# 한글 폰트 출력 가능 코드
import matplotlib.pyplot as plt
if platform.system() == "Darwin" :  # Mac 인 경우
    plt.rc('font', family = 'AppleGothic')
else :
    plt.rc('font', family = 'Malgun Gothic')

# 1. Simple data 형성

In [7]:
# 랜덤한 데이터 형성 (시드 부여)
np.random.seed(2022)     # seed 값을 2022로 고정

x = np.random.randint(1, 50, 30)    # 1부터 50까지 범위에서 무작위로 30개의 정수 생성
y = np.random.randint(1, 50, 30)

print(x)
print(y)

[29 46 49 49 25 19 25 17 42 34 28 12 20 31 41 12 49 20 39  9 15 30 17 27
 12 31 34  3 17 16]
[24 38 18 24 22 38 20 13 46 27 13 15 33 14 14 43  1 46 30 24 38 22 12 44
 47  3  1 37 47 42]


## 1-1. 데이터 시각화

In [3]:
# 데이터 프레임 형성
sample = pd.DataFrame(list(zip(x,y)), columns=['x', 'y'])   # zip()으로 묶고 list()로 리스트 변환

# 상관분석에 앞서 먼저 시각화를 이용하여 상관성 확인

sns.pairplot(sample, kind ='reg')
plt.show()

NameError: name 'pd' is not defined

## 1-2. 상관분석

### A. 공분산

In [10]:
# 하나씩 시행해보기

# mu = mean (공통적으로 적용)
mu_1 = np.sum(sample['x']/len(sample['x']))
mu_2 = np.sum(sample['y']/len(sample['y']))

# var = variance (잔차의 제곱합의 평균), 표준편차 = var의 제곱근
var_1 = np.sum([pow(i - mu_1, 2) for i in sample['x']]) / (len(sample['x']))
var_2 = np.sum([pow(i - mu_2, 2) for i in sample['y']]) / (len(sample['y']))
std_1 = var_1 ** 0.5
std_2 = var_2 ** 0.5

# 공분산 (표본상관계수를 구하는 것이 아니고 모집단으로 도출하므로 자유도는 0으로 설정)
covariance = sum((sample['x']-mu_1) * (sample['y'] - mu_2))/(len(sample['x']))

# COV(X,Y) : 모집단인 경우 (numpy와 비교, 자유도 0 설정은 ddof=0)
print(covariance)
print(np.cov(sample['x'], sample['y'], ddof=0)) # 공분산 행렬의 (1,2), (2,1)이 cov(x,y)

-61.68666666666666
[[160.90666667 -61.68666667]
 [-61.68666667 200.91555556]]


B. 피어슨 상관계수

In [11]:
# 위의 공분산에서 이어짐
correlation = covariance / (std_1 * std_2)

# 피어슨 상관계수
print(correlation)

-0.34308166001608753


In [12]:
# 피어슨 상관계수를 구하는 다른 방법으로는 scipy라이브러리를 활용하는 법

from scipy.stats import pearsonr
# 귀무가설은 '두 데이터의 상관 계수가 0이다. (독립)'로 설정되어 있음.
pearsonr(sample['x'], sample['y'])

PearsonRResult(statistic=-0.3430816600160875, pvalue=0.06344285692397392)

In [14]:
# 함수화 시행
def cov_pearson(data1, data2, population = True):
    # 라이브러리 호출
    import numpy as np
    from scipy.stats import pearsonr

    # 변수를 설정하는 부분입니다.
    mu_1 = np.sum(data1)/len(data1)
    mu_2 = np.sum(data2)/len(data2)
    var_1 = np.sum([pow(i - mu_1, 2) for i in data1]) / (len(data1))
    var_2 = np.sum([pow(i - mu_2, 2) for i in data2]) / (len(data2))
    std_1 = var_1 ** 0.5
    std_2 = var_2 ** 0.5
    
    # 데이터가 모수인지 표본인지에 따라 자유도를 산입할지 여부를 결정합니다.
    if population == True:
        covariance = sum( (data1 - mu_1) * (data2 - mu_2) ) / (len(data1))
        correlation = covariance / (std_1 * std_2)
        print(f'모집단 data 각각의 분산 값은 COV(X,X) = {round(var_1,3)}, COV(Y,Y) = {round(var_2,3)}(으)로 계산됩니다.')
        print(f'모집단 data에 대한 공분산 값 : {round(covariance,3)}')
        print(f'모집단 data에 대한 피어슨 상관계수 값 : {round(correlation,3)}')
    else:
        var_1 = np.sum([pow(i - mu_1, 2) for i in data1]) / (len(data1) - 1)
        var_2 = np.sum([pow(i - mu_2, 2) for i in data2]) / (len(data2) - 1)
        std_1 = var_1 ** 0.5
        std_2 = var_2 ** 0.5
        covariance_ddof = sum( (data1 - mu_1) * (data2 - mu_2) ) / (len(data1) -1)
        correlation_ddof = covariance_ddof / (std_1 * std_2)
        print(f'표본 data 각각의 추정된 분산 값은 COV(x,x) = {round(var_1,3)}, COV(y,y) = {round(var_2,3)}(으)로 계산됩니다.')
        print(f'표본 data로 추정한 모집단의 공분산 값 : {round(covariance_ddof,3)}')
        print(f'표본 data로 추정한 모집단의 피어슨 상관계수 값 : {round(correlation_ddof,3)}')

    # Scipy 라이브러리에서 구한 피어슨 계수를 통해 위에서 확인한 상관계수 값을 검증해봅니다.
    print(f'Scipy로 구한 피어슨 상관계수는 {round(pearsonr(data1, data2)[0], 3)}입니다.')
    
    # Scipy에서 제공하는 피어슨 검정의 P값을 이용하여 데이터 간의 독립성에 대한 검정 결과 해석을 출력합니다.
    if pearsonr(data1, data2)[1] < 0.05:
        print(f'Scipy는 각 데이터의 상관계수가 유의한지 판단합니다. 위 검정의 p-value는 {round(pearsonr(data1, data2)[1],5) * 100}%로 두 데이터의 연관성이 없다는 귀무가설을 5%의 유의수준으로 기각할 수 있습니다.')
    else:
        print(f'Scipy는 각 데이터의 상관계수가 유의한지 판단합니다. 위 검정의 p-value는 {round(pearsonr(data1, data2)[1],5) * 100}%로 두 데이터의 연관성이 없다는 귀무가설을 5%의 유의수준으로 기각할 수 없습니다.')

print('1. 모집단으로 가정한 경우')
cov_pearson(sample['x'], sample['y'], True)

print('\n2. 표본집단으로 가정한 경우')
cov_pearson(sample['x'], sample['y'], False)

1. 모집단으로 가정한 경우
모집단 data 각각의 분산 값은 COV(X,X) = 160.907, COV(Y,Y) = 200.916(으)로 계산됩니다.
모집단 data에 대한 공분산 값 : -61.687
모집단 data에 대한 피어슨 상관계수 값 : -0.343
Scipy로 구한 피어슨 상관계수는 -0.343입니다.
Scipy는 각 데이터의 상관계수가 유의한지 판단합니다. 위 검정의 p-value는 6.343999999999999%로 두 데이터의 연관성이 없다는 귀무가설을 5%의 유의수준으로 기각할 수 없습니다.

2. 표본집단으로 가정한 경우
표본 data 각각의 추정된 분산 값은 COV(x,x) = 166.455, COV(y,y) = 207.844(으)로 계산됩니다.
표본 data로 추정한 모집단의 공분산 값 : -63.814
표본 data로 추정한 모집단의 피어슨 상관계수 값 : -0.343
Scipy로 구한 피어슨 상관계수는 -0.343입니다.
Scipy는 각 데이터의 상관계수가 유의한지 판단합니다. 위 검정의 p-value는 6.343999999999999%로 두 데이터의 연관성이 없다는 귀무가설을 5%의 유의수준으로 기각할 수 없습니다.
